<h1><center> Assignment 3 </center></h1>

Bruno Gomes Coelho - bruno.gomes.coelho@usp.br



- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

<h1> <center> Machine Learning & NN </center> </h1>

## a) Adam optimizer  

### i) Briefly explain (you don’t need to prove mathematically, just give an intuition) how using **m** stops the updates from varyingas much and why this low variance may be helpful to learning, overall.

- Helps with local minima
- Helps with saddle points
- "Cancelling" zig zags
- Closer to the true gradiente

### ii) Why (does dividing by $ \sqrt{v} $)  help with learning?

- Smaller gradientes will will pick up, proportionally, more velocity -> will help with "flat" terrain;
- In general, the magnitude of the updates are invariant to scaling -> we control using just the lr 


## b) Dropout

### i) What must γ equal in terms of $ p_{drop} $ ?

γ = $ \frac{1}{1 - p_{drop}} $

This is basically the cs231n dropout trick better explained :)

### ii) Why should we apply dropout during training but not during evaluation?

We want **deterministic** test time

<h1> <center> Neural Transition-Based Dependecy Parsing</center> </h1>

## a)  Dependence parse

# ![image](2a.jpg)

## b)  A sentence containing n words will be parsed in how many steps (in terms of n)?

2n - A cada iteração ou colocamos na stack ou removemos do buffer

<h3><center> parser_transition.py </center></h3>

In [ ]:
def __init__():
    self.stack = ["ROOT"]
    self.buffer = [x for x in self.sentence]
    self.dependencies = []


In [1]:
def parse_step(self, transition):
    ### YOUR CODE HERE (~7-10 Lines)
    ### TODO:
    ###     Implement a single parsing step, i.e. the logic for the following as
    ###     described in the pdf handout:
    ###         1. Shift
    ###         2. Left Arc
    ###         3. Right Arc

    # Helper variables for readibility
    FIRST, SECOND = -1, -2
    get_remove = {"RA": FIRST, "LA": SECOND}
    get_head = {"LA": FIRST, "RA": SECOND}

    if transition == "S":
        word = self.buffer.pop(0)
        self.stack.append(word)

    if transition in ["LA", "RA"]:
        head = self.stack[get_head[transition]]
        remove = self.stack.pop(get_remove[transition])
        self.dependencies.append((head, remove)) 

NameError: name 'transition' is not defined

In [3]:
def minibatch_parse(sentences, model, batch_size):

    dependencies = []
    ### YOUR CODE HERE (~8-10 Lines)
    ### TODO:
    ###     Implement the minibatch parse algorithm as described in the pdf handout
    ###
    ###     Note: A shallow copy (as denoted in the PDF) can be made with the "=" sign in python, e.g.
    ###                 unfinished_parses = partial_parses[:].

    partial_parses = [PartialParse(sent) for sent in sentences]
    unfinished_parses = partial_parses[:]

    while unfinished_parses:
        minibatch = unfinished_parses[:batch_size]
        transitions = model.predict(minibatch)

        for partial_parser, transation in zip(minibatch, transitions):
            partial_parser.parse_step(transation)
            if (not partial_parser.buffer) and len(partial_parser.stack) == 1:
                unfinished_parses.remove(partial_parser)

    return [pp.dependencies for pp in partial_parses]


<h3><center> parser_model.py </center></h3>

In [5]:
def __init__(self, embeddings, n_features=36,
             hidden_size=200, n_classes=3, dropout_prob=0.5):
    ### YOUR CODE HERE (~5 Lines)
    ### TODO:
    ###     1) Construct `self.embed_to_hidden` linear layer, initializing the weight matrix
    ###         with the `nn.init.xavier_uniform_` function with `gain = 1` (default)
    ###     2) Construct `self.dropout` layer.
    ###     3) Construct `self.hidden_to_logits` linear layer, initializing the weight matrix
    ###         with the `nn.init.xavier_uniform_` function with `gain = 1` (default)
    ###
    self.embed_to_hidden = nn.Linear(self.embed_size*self.n_features, 
                                     self.hidden_size)
    nn.init.xavier_uniform_(self.embed_to_hidden.weight, gain=1)

    self.dropout = nn.Dropout(self.dropout_prob)

    self.hidden_to_logits = nn.Linear(self.hidden_size, self.n_classes)
    nn.init.xavier_uniform_(self.hidden_to_logits.weight, gain=1)

In [ ]:
def embedding_lookup(self, t):
    ### YOUR CODE HERE (~1-3 Lines)
    ### TODO:
    ###     1) Use `self.pretrained_embeddings` to lookup the embeddings for the input tokens in `t`.
    ###     2) After you apply the embedding lookup, you will have a tensor shape (batch_size, n_features, embedding_size).
    ###         Use the tensor `view` method to reshape the embeddings tensor to (batch_size, n_features * embedding_size)
    ###
    ### Note: In order to get batch_size, you may need use the tensor .size() function:
    ###         https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size
    x = self.pretrained_embeddings(t)
    x = x.view(x.size()[0], -1)

    ### END YOUR CODE
    return x

In [6]:
def forward(self, t):
    ###  YOUR CODE HERE (~3-5 lines)
    ### TODO:
    ###     1) Apply `self.embedding_lookup` to `t` to get the embeddings
    ###     2) Apply `embed_to_hidden` linear layer to the embeddings
    ###     3) Apply relu non-linearity to the output of step 2 to get the hidden units.
    ###     4) Apply dropout layer to the output of step 3.
    ###     5) Apply `hidden_to_logits` layer to the output of step 4 to get the logits.
    ###
    ### Note: We do not apply the softmax to the logits here, because
    ### the loss function (torch.nn.CrossEntropyLoss) applies it more efficiently.
    ###
    ### Please see the following docs for support:
    ###     ReLU: https://pytorch.org/docs/stable/nn.html?highlight=relu#torch.nn.functional.relu

    x = self.embedding_lookup(t)
    x = self.embed_to_hidden(x)
    x = nn.functional.relu(x)
    x = self.dropout(x)
    logits = self.hidden_to_logits(x)

    ### END YOUR CODE
    return logits

<h3><center> run.py </center></h3>

In [ ]:
def train(parser, train_data, dev_data, output_path, 
          batch_size=1024, n_epochs=10, lr=0.0005):
    ### YOUR CODE HERE (~2-7 lines)
    ### TODO:
    ###      1) Construct Adam Optimizer in variable `optimizer`
    ###      2) Construct the Cross Entropy Loss Function in variable `loss_func`
    ###

    optimizer = optim.Adam(parser.model.parameters())
    loss_func = nn.CrossEntropyLoss()

    ### END YOUR CODE

In [7]:
def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size):
    parser.model.train() # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        for i, (train_x, train_y) in enumerate(minibatches(train_data, batch_size)):
            optimizer.zero_grad()   # remove any baggage in the optimizer
            loss = 0. # store loss for this batch here
            train_x = torch.from_numpy(train_x).long()
            train_y = torch.from_numpy(train_y.nonzero()[1]).long()

            ### YOUR CODE HERE (~5-10 lines)
            ### TODO:
            ###      1) Run train_x forward through model to produce `logits`
            ###      2) Use the `loss_func` parameter to apply the PyTorch CrossEntropyLoss function.
            ###         This will take `logits` and `train_y` as inputs. It will output the CrossEntropyLoss
            ###         between softmax(`logits`) and `train_y`. Remember that softmax(`logits`)
            ###         are the predictions (y^ from the PDF).
            ###      3) Backprop losses
            ###      4) Take step with the optimizer
            ### Please see the following docs for support:
            ###     Optimizer Step: https://pytorch.org/docs/stable/optim.html#optimizer-step
            logits = model.forward(train_x)
            loss = loss_func(logits, train_y)
            loss.backward()
            optimizer.step()

Resultado final: 
- test UAS: 89.02                                                  



In [9]:
### f) Different parsing mistakes


![image](2f1.jpg)

Verbal Phrase
Heading -> Fearing

![image](2f2.jpg)

Coordination Attachment
rush -> rescue

![image](2f3.jpg)

Prepositional Phrase
guy -> Midland

![image](2f4.jpg)

Modifier Attachment
crucial -> most

<center> Thanks :) </center> 